In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
s0 = 1000
r = 0
T = 1 / 12
K = 1000
xi = 0.6
# symbolic variables
s, v, t, v0 = symbols('s v t v0')
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + v0 / 2) * t) / sqrt(v0 * t)
delta0 = 0.5 * (v - v0) * s * density(N)(d_p) / sqrt(v0 * t)

def bs(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d_m = d_p - sigma * np.sqrt(T)
  return S * N(d_p) - K * np.exp(-r * T) * N(d_m)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

delta = [delta0]
delta_func = []
for _ in range(1, 5):
  delta.append(L(delta[-1]))
for i in range(0, 5):
  delta_func.append(lambdify([s, v, t, v0], delta[i]))

V = np.arange(0.1, 1.2, 0.1)
Approx_4 = []
for v in V:
  p0_approx = bs(s0, K, T, r, np.sqrt(v)) + T * delta_func[0](s0, v, T, v)
  p1_approx = p0_approx + T**2 / 2 * delta_func[1](s0, v, T, v)
  p2_approx = p1_approx + T**3 / 6 * delta_func[2](s0, v, T, v)
  p3_approx = p2_approx + T**4 / 24 * delta_func[3](s0, v, T, v)
  p4_approx = p3_approx + T**5 / 120 * delta_func[4](s0, v, T, v)
  Approx_4.append(p4_approx)

Monte_carlo = np.array([36.0006, 50.7456, 62.0950, 71.6717, 80.1100, 87.7368, 94.7477, 101.2699, 107.3927, 113.1820, 118.6858])
JFE_results = np.array([36.4457, 51.3208, 62.7745, 72.4363, 80.9467, 88.6366, 95.7032, 102.2754, 108.4427, 114.2706, 119.8085])
JFE_results_diff = np.array([1.2364, 1.1334, 1.0943, 1.0668, 1.0445, 1.0256, 1.0085, 0.9929, 0.9777, 0.9618, 0.9460])
Approx_diff = (Approx_4 - Monte_carlo) / Monte_carlo * 100

df = pd.DataFrame({"V": V, "Monte carlo": Monte_carlo, "Replicate price": np.round(Approx_4, 4), "Replicate %diff": np.round(Approx_diff, 4), "JFE price": JFE_results, "JFE %diff": JFE_results_diff})
print(df)
df.to_csv("Table 2 Panel B.csv")

      V  Monte carlo  Replicate price  Replicate %diff  JFE price  JFE %diff
0   0.1      36.0006          36.6167           1.7114    36.4457     1.2364
1   0.2      50.7456          51.5021           1.4907    51.3208     1.1334
2   0.3      62.0950          62.9573           1.3886    62.7745     1.0943
3   0.4      71.6717          72.6188           1.3215    72.4363     1.0668
4   0.5      80.1100          81.1286           1.2715    80.9467     1.0445
5   0.6      87.7368          88.8177           1.2320    88.6366     1.0256
6   0.7      94.7477          95.8836           1.1988    95.7032     1.0085
7   0.8     101.2699         102.4550           1.1702   102.2754     0.9929
8   0.9     107.3927         108.6217           1.1444   108.4427     0.9777
9   1.0     113.1820         114.4490           1.1194   114.2706     0.9618
10  1.1     118.6858         119.9864           1.0958   119.8085     0.9460
